In [1]:
## Import libraries
import pandas as pd
import numpy as np

In [2]:
'''
function name: parse_df_columns

purpose: parsing the last aggregate column of the gtf/gff3 into useful columns and cleaning non-relevant columns

input: dataframe containining "raw" gtf/gff

output: dataframe containing gtf with useful columns ["gene_id", "transcript_id", etc...]
'''

def parse_df_columns(df, is_ref=True):

    if is_ref:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split("source_gene=", expand=True)[1].str.split(';', expand=True)[0]

        ## Get transcript ids
        df["transcript_id"] = df["other"].str.split("source_transcript=", expand=True)[1].str.split(';', expand=True)[0]

        ## Get CHM gene_ids
        df["CHM_gene_id"] = df["other"].str.split("gene_id=", expand=True)[1].str.split(';', expand=True)[0]

        ## Get transcript ids
        df["CHM_transcript_id"] = df["other"].str.split("transcript_id=", expand=True)[1].str.split(';', expand=True)[0]
        
        ## Get transcript names
        df["transcript_name"] = df["other"].str.split("source_transcript_name=", expand=True)[1].str.split(';', expand=True)[0]
        
        ## Get gene names
        df["gene_name"] = df["other"].str.split("source_gene_common_name=", expand=True)[1].str.split(';', expand=True)[0]
        
        ## Get start codon
        df["start_codon"] = df["other"].str.split("adj_start=", expand=True)[1].str.split(";", expand=True)[0]
        
        ## Get stop codon
        df["stop_codon"] = df["other"].str.split("adj_stop=", expand=True)[1].str.split(";", expand=True)[0]        

        ## Only keep relevant
        df = df[["chr", "start", "end", "strand", "type", "gene_id", "transcript_id", "CHM_gene_id",
                 "CHM_transcript_id", "transcript_name", "gene_name", "start_codon", "stop_codon"]].copy()

        ## Drop duplicates
        df.drop_duplicates(inplace=True)
        

    else:

        ## Get CHM gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)

        ## Get CHM transcript ids
        df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]

        ## Get exon number
        df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Label novel transcripts
        df.loc[df["transcript_id"].str.startswith("tx."), "is_novel_transcript"] = True
        df.loc[~df["transcript_id"].str.startswith("tx."), "is_novel_transcript"] = False

        ## Label novel genes
        df.loc[df["gene_id"].str.startswith("gene."), "is_novel_gene"] = True
        df.loc[~df["gene_id"].str.startswith("gene."), "is_novel_gene"] = False

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)

    for col in df.columns:
        df.loc[df[col].isnull(), col] = np.NaN
        

    return df

In [3]:
'''
function name: merge_annotations

purpose: Merge useful/relevant information from both annotations while removing repeated and irrelevant information

input: Two different GTF annotations

output: One GTF annotation containing all the relevant information
'''

def merge_annotations(ref_gtf, bambu_gtf):
    
    ## Merge the two annotations
    names_ref_gtf = ref_gtf[["transcript_id", "gene_id", "gene_name", "transcript_name"]].copy()
    merged_gtf = pd.merge(bambu_gtf, names_ref_gtf, on=['gene_id', 'transcript_id'], how='left')
    merged_gtf.drop_duplicates(inplace=True)

    ## Label novel transcripts
    merged_gtf.loc[merged_gtf["transcript_id"].str.startswith("tx."), "is_novel_transcript"] = True
    merged_gtf.loc[~merged_gtf["transcript_id"].str.startswith("tx."), "is_novel_transcript"] = False

    ## Label novel genes
    merged_gtf.loc[merged_gtf["gene_id"].str.startswith("gene."), "is_novel_gene"] = True
    merged_gtf.loc[~merged_gtf["gene_id"].str.startswith("gene."), "is_novel_gene"] = False

    ## Create temporary variable only containing novel transcripts
    temp = merged_gtf.loc[merged_gtf["is_novel_transcript"] == True]

    ## Annotate novel transcripts
    merged_tmp = pd.merge(temp, ref_gtf[["gene_id", "gene_name"]], on=['gene_id'], how='left')
    merged_tmp.drop_duplicates(inplace=True)
    merged_tmp["gene_name"] = merged_tmp["gene_name_y"]
    merged_tmp.drop(columns=["source", "gene_name_y", "gene_name_x"], inplace=True)

    ## Return novel transcripts to original annotation
    merged_final = pd.merge(merged_gtf, merged_tmp, on=['chr', 'type', 'start', 'end', 'strand', 'transcript_id',
                    'transcript_name', 'gene_id', 'is_novel_transcript', 'is_novel_gene', 'exon_number'], how="left")

    ## Get gene names for novel transcripts of known genes
    merged_final.gene_name_x.fillna(merged_final.gene_name_y, inplace=True)
    merged_final["gene_name"] = merged_final["gene_name_x"]
    merged_final.drop(columns =["gene_name_x", "gene_name_y"], inplace=True)
    
    ## Get start and stop codons for known transcripts and exons of protein coding genes
    ref_gtf = ref_gtf[["chr", "type", "start", "end", "strand", "transcript_id", "gene_id", "start_codon", "stop_codon"]]
    merged_final = pd.merge(merged_final, ref_gtf, on=["chr", "type", "start", "end", "strand", "transcript_id", "gene_id"], how="left")
    
    ## Only keep relevant columns
    merged_final = merged_final[["chr", "type", "start", "end", "strand", "transcript_id", "gene_id", "gene_name", 
                    "exon_number", "transcript_name", "start_codon", "stop_codon", "is_novel_gene", "is_novel_transcript"]]


    
    return merged_final 

In [4]:
'''
function name: calculate_cpm

purpose: calculate cpm from bambu counts file and return it

input: Bambu transcript counts file

output: Same file with cpm instead of counts
'''

def calculate_cpm(counts):
    
    ## Get the names of the count columns
    list_counts_cols = counts.columns[2:]
    
    ## Loop through counts columns
    for col in list_counts_cols:
        
        ## Get sum of counts column
        sum_col = counts[col].sum()
        
        ## Change name of counts column to CPM
        new_col_name = col[0:(len(col)-7)] + "_CPM"
        
        ## Calculate CPM for new CPM column
        counts[new_col_name] = ((counts[col]/sum_col) * 1000000)
        
    ## Drop old Columns
    counts.drop(columns=list_counts_cols, inplace=True)
    
    return counts

In [5]:
'''
name: make_gene_and_transcript_converter

input: The CHM13 CAT/Liftoff gff annotation version 2.0

output: A dataframe with ["gene_id", "transcript_id", "gene_name", "transcript_name"] formatted in the same way as the 
bambu reference, so that we can properly assign gene and transcript names.

purpose: Creating a list that allows us to assign transcript and gene names to the bambu annotation based on the transcript
id and gene ID
'''


def make_gene_and_transcript_converter(gff):
    
    ## Change name of duplicate Ensembl IDs to CHM IDs
    gff.loc[gff["transcript_id"] == "N/A", "transcript_id"] = gff["CHM_transcript_id"]
    gff_transcripts = gff.loc[gff["type"] == "transcript"].copy()
    gff_transcripts = gff_transcripts[["transcript_id", "CHM_transcript_id"]].drop_duplicates()
    gff_transcripts = gff_transcripts[gff_transcripts['transcript_id'].duplicated() == True]
    dup_trans = gff_transcripts["transcript_id"].dropna().values.tolist()
    gff.loc[gff["transcript_id"].isin(dup_trans), "transcript_id"] = gff["transcript_id"] + "(" + gff["CHM_transcript_id"] + ")"

    ## Change name of duplicate gene ids to CHM ids
    gff.loc[gff["gene_id"] == "None", "gene_id"] = gff["CHM_gene_id"]
    gff_genes = gff.loc[gff["type"] == "transcript"].copy()
    gff_genes = gff_genes[["gene_id", "CHM_gene_id"]].drop_duplicates()
    gff_genes = gff_genes[gff_genes['gene_id'].duplicated() == True]
    dup_genes = gff_genes["gene_id"].dropna().values.tolist()
    gff.loc[gff["gene_id"].isin(dup_genes), "gene_id"] = gff["gene_id"] + "(" + gff["CHM_gene_id"] + ")"

    
    ## Fix gene names for MSTRG Genes
    gff_names = gff.loc[gff["type"] == "transcript"].copy()
    gff_names = gff_names[["gene_id", "gene_name", "CHM_gene_id"]].copy()
    gff_names.loc[gff_names["gene_name"].str.contains("MSTRG."), "gene_name"] = np.NaN
    gff_names.dropna(inplace=True)
    gff_names["gene_name"] = gff_names["gene_name"] + "(" + gff_names["CHM_gene_id"] + ")"
    gff_names.drop(columns="CHM_gene_id", inplace=True)
    gff_names.drop_duplicates(inplace=True, subset=["gene_id"])
    gff.drop(columns="gene_name", inplace=True)
    gff = pd.merge(gff, gff_names, on="gene_id", how="left")
    gff = gff[["gene_id", "transcript_id", "gene_name", "transcript_name",
              "start", "end", "type", "start_codon", "stop_codon", "chr", "strand"]].copy()


    return gff

In [6]:
'''
name: make_merged_exons_table

input: dataframe with exons file ["chr", "start", "end", "gene_id", "gene_name", "strand"]

output: The same dataframe, but the exons that have overlapping positions are merged

purpose: Creating a dataframe with merged exons
'''

def make_merged_exons_table(df):
    
    ## Drop duplicates and create a list with all gene ids
    df.drop_duplicates(inplace=True)
    list_gene_ids = df["gene_id"].unique()
    
    ## Create empty list to hold final results
    final_merged_exons_list = []

    ## Loop through gene ids
    for id in list_gene_ids:
        
        ## Create dataframe with only that gene ID and sort it by start position
        df_gene = df.loc[df["gene_id"] == id].copy()
        df_gene.sort_values(by='start', inplace=True)
        
        ## Create exon list flattened to 1D
        exon_list = df_gene.to_numpy()
        exon_list = exon_list.flatten()
        exon_list = exon_list.tolist()
        
        ## Start merged exon list
        merged_exon_list = exon_list[0:6]

        ## Loop through exon list  starting by the second entry, every entry is 6 long
        for j in range(12, (len(exon_list) + 6), 6):
            
            ## If exons overlap (End >= Start on next exon)
            if merged_exon_list[(len(merged_exon_list) - 4)] >= (exon_list[j - 5] - 1):
                
                ## If merged exon does not fully envelop the next exon, then make change the end position
                if merged_exon_list[(len(merged_exon_list) - 4)] < exon_list[j - 4]:
                    merged_exon_list[(len(merged_exon_list) - 4)] = exon_list[j - 4]
            
            ## If exons don't overlap add the exon to the list
            else:
                merged_exon_list.extend(exon_list[(j-6):j])
        
        ## Once looped through the whole list merge it to the final merged exons list
        final_merged_exons_list.extend(merged_exon_list)

    ## Create real final list
    real_final_list = []

    ## Transform 1D final_merged_exon_list into 2D list
    for i in range(6, (len(final_merged_exons_list) - 6), 6):
        current_list = final_merged_exons_list[i: i + 6]
        real_final_list.append(current_list)



    ## Transform real_final_list into a dataframe and return it
    df_merged_exons = pd.DataFrame(real_final_list,
                                   columns =['chr', "start", "end", "gene_id", "gene_name", "strand"])
    
    return df_merged_exons

In [7]:
'''
name: find_novel_exons_and_junctions

input: dataframe with exons annotation.

output: The same dataframe, but the exons flags indicating whether the exon is novel and whether the junction is novel.

purpose: Creating flags that indicate whether exons or exon junctions are novel.
'''

def find_novel_exons_and_junctions(df_test):
    
    ## Create unique exon ids
    df_test["unique_exon_id"] = df_test["start"].astype(str) + "-" + df_test["end"].astype(str) + ":" + df_test["gene_id"]

    ## Create a list with all the unique exons ids for all known transcripts
    df_known_exons = df_test.loc[df_test["is_novel_transcript"] == False]["unique_exon_id"].unique()

    ## If the exon ID is contained in the known exons IDs then the exon is not new, otherwise it is
    df_test.loc[df_test["unique_exon_id"].isin(df_known_exons), "is_novel_exon"] = False
    df_test.loc[~df_test["unique_exon_id"].isin(df_known_exons), "is_novel_exon"] = True

    ## Drop the unique exon id column and set df_known_exons to None to free up memory
    df_test.drop(columns=["unique_exon_id"], inplace=True)
    df_known_exons = None

    ## Sort test dataframe by transcript_id and start position
    df_test.sort_values(by=["transcript_id", "start"], inplace=True)

    ## Create an empty dataframe we can concatenate things into
    df_final = df_test.iloc[0:0].copy()

    ## Create numpy arrays for data we will be messing with
    transcript_ids = df_test['transcript_id'].copy().to_numpy()
    gene_ids = df_test['gene_id'].copy().to_numpy()
    start = df_test['start'].copy().astype(str).to_numpy()
    end = df_test['end'].copy().astype(str).to_numpy()

    unique_junction_id_list = np.empty((len(gene_ids)), dtype=object)

    ## loop through data and generate unique junction IDs
    for i in range(len(gene_ids)):

        if i < (len(gene_ids) - 1):

            if (transcript_ids[i] == transcript_ids[i+1]):
                unique_transcript_id = end[i] + "-" + start[i+1] +':' + gene_ids[i]
                unique_junction_id_list[i] = unique_transcript_id

            else:
                unique_junction_id_list[i] = np.NaN
        else:
            unique_junction_id_list[i] = np.NaN
    
    ## Set a column in df_test equal to unique_junction_id_list
    df_test["unique_junction_id"] = unique_junction_id_list
    
    ## Create series with known junction ids
    df_known_junctions = df_test.loc[df_test["is_novel_transcript"] == False]["unique_junction_id"].unique()

    ## If an exon has a known junction it is not new, otherwise it is
    df_test.loc[df_test["unique_junction_id"].isin(df_known_junctions), "is_novel_junction"] = False
    df_test.loc[~df_test["unique_junction_id"].isin(df_known_junctions), "is_novel_junction"] = True
    df_test.loc[df_test["unique_junction_id"].isna(), "is_novel_junction"] = False

    ## Drop useless columns and free up memory
    df_test.drop(columns=["unique_junction_id"], inplace=True)
    df_known_exons = None
    
    return df_test

In [9]:
def main():
    
    ## Import reference GTF, and Create a copy of the original reference CHM13 gff for late user 
    ref_gff = pd.read_csv("../../cDNA-comparison/article_analysis/annotations/CHM13.v2.0.gff3", delimiter="\t", header=1, low_memory=False,
                     names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

    ## Only keep transcripts and exons, and parse through dataframe to get gene and transcript names.
    ref_gff = ref_gff.loc[ref_gff["type"].isin(["transcript", "exon"])]
    ref_gff = parse_df_columns(ref_gff, is_ref=True)


    ## Create gene/transcript converter file base on reference genome
    ref_gff = make_gene_and_transcript_converter(ref_gff)

    ## Import Bambu GTF including novel genes/transcripts. Parse to data to extract gene_id and transcript_id
    bambu_gtf = pd.read_csv("../novel_gene_body_raw_data/chm13_bambu/extended_annotations.gtf", header=None, delimiter="\t", low_memory=False,
                           names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

    bambu_gtf = bambu_gtf.loc[bambu_gtf["type"].isin(["exon", "transcript"])]
    bambu_gtf = parse_df_columns(bambu_gtf, is_ref=False)

    ## Create merged annotation
    merged_gtf = merge_annotations(ref_gff, bambu_gtf)
    
    ## Import bambu counts file, COLUMN NAMES ARE HARD CODED< PAY ATTENTION, ORDER COULD CHANGE!
    bambu_counts = pd.read_csv("../novel_gene_body_raw_data/chm13_bambu/counts_transcript.txt", delimiter="\t", low_memory=False, header=0,
                          names=["transcript_id", "gene_id", 'sample_PAM54902', 'sample_PAM54401',
                                                              'sample_PAM54335', 'sample_PAM54788', 'sample_PAK20814'])

    ## Calculate cpm for bambu counts file
    bambu_counts = calculate_cpm(bambu_counts)

    ## Create annotation with only transcript, add the bambu cpm to the annotation, and save it
    transcripts = merged_gtf.loc[merged_gtf["type"] == "transcript"].copy()
    transcripts = pd.merge(transcripts, bambu_counts, on=["transcript_id", "gene_id"], how="left")
    transcripts.drop(columns=["exon_number"], inplace=True)
    transcripts.to_csv("../novel_gene_body_processed_data/2022-06-08_bah_transcript_annotation_and_cpm_cDNA.csv", index=False)

    ## Create exon only annotation and save it
    exons = merged_gtf.loc[merged_gtf["type"] == "exon"].copy()
    exons_final = find_novel_exons_and_junctions(exons)
    exons_final.to_csv("../novel_gene_body_processed_data/2022-06-08_bah_exon_annotation_cDNA.csv", index=False)

    ## Create a bed file with only exons to later make the merged exons annotation using bedtools
    exons_bedfile = exons_final[['chr', 'start', 'end', 'gene_id', 'gene_name', 'strand']].copy()
    exons_merged_file = make_merged_exons_table(exons_bedfile)
    exons_merged_file.to_csv("../novel_gene_body_processed_data/2022-06-08_bah_merged_exons_cDNA.csv", index=False)
    
main()

In [12]:
df = pd.read_csv("../novel_gene_body_processed_data/2022-06-08_bah_exon_annotation_cDNA.csv", low_memory=False)

In [13]:
df_novel = df.loc[df["is_novel_transcript"] == True]

In [17]:
df_novel["gene_id"].nunique()

621

In [18]:
df_novel.tail()

,chr,type,start,end,strand,transcript_id,gene_id,gene_name,exon_number,transcript_name,start_codon,stop_codon,is_novel_gene,is_novel_transcript,is_novel_exon,is_novel_junction
1446945,chr2,exon,103302104.0,103302530.0,+,tx.97,ENSG00000170417.16,TMEM182(CHM13_G0031823),5,NaN,NaN,NaN,False,True,True,False
1446946,chr2,exon,105314453.0,105314750.0,+,tx.98,ENSG00000198914.5,POU3F3(CHM13_G0031856),1,NaN,NaN,NaN,False,True,True,True
1446947,chr2,exon,105387145.0,105388956.0,+,tx.98,ENSG00000198914.5,POU3F3(CHM13_G0031856),2,NaN,NaN,NaN,False,True,True,False
1446948,chr2,exon,105330404.0,105331134.0,-,tx.99,ENSG00000229743.4,LINC01159(CHM13_G0031857),2,NaN,NaN,NaN,False,True,False,True
1446949,chr2,exon,105333705.0,105333802.0,-,tx.99,ENSG00000229743.4,LINC01159(CHM13_G0031857),1,NaN,NaN,NaN,False,True,True,False


In [19]:
bambu_counts = pd.read_csv("../novel_gene_body_raw_data/chm13_bambu/counts_transcript.txt", delimiter="\t", low_memory=False, header=0,
                          names=["transcript_id", "gene_id", 'sample_PAM54902', 'sample_PAM54401',
                                                              'sample_PAM54335', 'sample_PAM54788', 'sample_PAK20814'])

In [21]:
bambu_counts_novel = bambu_counts.loc[bambu_counts["transcript_id"].str.startswith("tx.")]

In [22]:
## Filter novel genes and transcripts, must be present in every sample with 5+ counts.
count_cols = ['sample_PAM54902', 'sample_PAM54401', 'sample_PAM54335', 'sample_PAM54788', 'sample_PAK20814']

for col in count_cols:
    bambu_counts_novel = bambu_counts_novel.loc[bambu_counts_novel[col] > 5]

In [23]:
df_novel = df_novel.loc[df_novel["transcript_id"].isin(bambu_counts_novel["transcript_id"])]

In [28]:
df_novel["is_novel_junction"].value_counts()

False    1515
True      668
Name: is_novel_junction, dtype: int64

In [29]:
df_novel["is_novel_exon"].value_counts()

True     1185
False     998
Name: is_novel_exon, dtype: int64

In [32]:
df_novel.loc[((df_novel["is_novel_junction"] == False) & (df_novel["is_novel_exon"] == True))]

,chr,type,start,end,strand,transcript_id,gene_id,gene_name,exon_number,transcript_name,start_codon,stop_codon,is_novel_gene,is_novel_transcript,is_novel_exon,is_novel_junction
1444516,chr1,exon,9994993.0,9995515.0,+,tx.1,ENSG00000251503.8,CENPS-CORT(CHM13_G0000323),6,NaN,NaN,NaN,False,True,True,False
1444522,chr2,exon,113040841.0,113041602.0,+,tx.101,ENSG00000125611.15,CHCHD5(CHM13_G0032033),2,NaN,NaN,NaN,False,True,True,False
1444527,chr2,exon,114036198.0,114036977.0,-,tx.103,ENSG00000240356.6,RPL23AP7(CHM13_G0032075),3,NaN,NaN,NaN,False,True,True,False
1444529,chr2,exon,114051609.0,114051845.0,-,tx.103,ENSG00000240356.6,RPL23AP7(CHM13_G0032075),1,NaN,NaN,NaN,False,True,True,False
1444532,chr2,exon,119715925.0,119716307.0,-,tx.104,gene.185,NaN,1,NaN,NaN,NaN,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446940,chr2,exon,102082397.0,102082766.0,+,tx.96,gene.136,NaN,4,NaN,NaN,NaN,True,True,True,False
1446941,chr2,exon,103195595.0,103195693.0,+,tx.97,ENSG00000170417.16,TMEM182(CHM13_G0031823),1,NaN,NaN,NaN,False,True,True,False
1446945,chr2,exon,103302104.0,103302530.0,+,tx.97,ENSG00000170417.16,TMEM182(CHM13_G0031823),5,NaN,NaN,NaN,False,True,True,False
1446947,chr2,exon,105387145.0,105388956.0,+,tx.98,ENSG00000198914.5,POU3F3(CHM13_G0031856),2,NaN,NaN,NaN,False,True,True,False


In [38]:
df.loc[df["gene_id"] == "gene.185"]

,chr,type,start,end,strand,transcript_id,gene_id,gene_name,exon_number,transcript_name,start_codon,stop_codon,is_novel_gene,is_novel_transcript,is_novel_exon,is_novel_junction
1444530,chr2,exon,119714686.0,119715057.0,-,tx.104,gene.185,NaN,3,NaN,NaN,NaN,True,True,True,True
1444531,chr2,exon,119715305.0,119715421.0,-,tx.104,gene.185,NaN,2,NaN,NaN,NaN,True,True,True,True
1444532,chr2,exon,119715925.0,119716307.0,-,tx.104,gene.185,NaN,1,NaN,NaN,NaN,True,True,True,False
